In [1]:
import ast
import numpy as np
import pandas as pd

import os
import shutil
import tarfile

import cv2
import skimage
import scipy.ndimage
import napari

from scipy import ndimage
import mcubes
import trimesh
from scipy.ndimage import zoom
from plyfile import PlyData, PlyElement 

from auxiliary import values as v
from auxiliary.data.dataset_ht import HtDataset
from auxiliary.data import imaging

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
img_path = v.data_path + 'auxiliary/Jyothy_Test/em17.5.tif'
img = skimage.io.imread(img_path)

viewer, img_layer = napari.imshow(img, name='Image_r')

In [8]:
r, g, b = cv2.split(img)

g = cv2.normalize(g, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
g = cv2.bilateralFilter(g, 5, 75, 75)
g = (g / 255.0).astype(np.float32)

r = cv2.normalize(r, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
r = cv2.bilateralFilter(r, 5, 75, 75)
r = (r / 255.0).astype(np.float32)

pre_g_layer = viewer.add_image(g, name='Preprocessed Image g')
pre_b_layer = viewer.add_image(r, name='Preprocessed Image b')

In [4]:
# cv2.imshow('image', img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()



In [5]:
from nuclei_segmentation import my_cellpose as cp


model = cp.load_model(model_type='cyto3')

GPU activated: False
Loading model: cyto3


In [ ]:
masks, flows, styles, _ = model.eval(
    [g, r, None], 
    normalize=False,
    channels=[0, 1],
)

print(masks.shape, masks.dtype)
print(np.unique(masks))

In [ ]:
# cv2.imshow('image', np.hstack([img, masks]))
# cv2.waitKey(0)
# cv2.destroyAllWindows()

mask_layer = viewer.add_image(masks, name='Masks', blending='additive')